In [9]:
from dotenv import load_dotenv
load_dotenv()

True

### 2. Configuración e Importaciones

In [10]:
# @title 🔧 2. Configuración e Importaciones
import os
import json
import requests
from llama_index.core import Settings
from llama_index.core.tools import FunctionTool
#from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI as LlamaOpenAI
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct, Filter, FieldCondition, Range
from openai import OpenAI

In [11]:
from openai import OpenAI

# Obtener la API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

if not OPENAI_API_KEY:
    raise ValueError("❌ No se encontró OPENAI_API_KEY en el .env")

# Configuración Global
client = OpenAI(api_key=OPENAI_API_KEY) # Cliente nativo para embeddings manuales

#### Qdrant

In [12]:
from qdrant_client import QdrantClient

# Obtener valores
QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")

# Validar variables
if not QDRANT_URL:
    raise ValueError("❌ No se encontró QDRANT_URL en el .env")

if not QDRANT_API_KEY:
    raise ValueError("❌ No se encontró QDRANT_API_KEY en el .env")

qdrant_client = QdrantClient(
    url= QDRANT_URL, 
    api_key= QDRANT_API_KEY,
)

print(qdrant_client.get_collections())

COLLECTION_NAME = "metabolomics_agent_db"

print("✅ Configuración cargada.")

collections=[CollectionDescription(name='bio_semantic_db')]
✅ Configuración cargada.


### 3. Ingesta con Semantic Chunking

In [14]:
# @title 🧠 3. Ingesta con Semantic Chunking (La parte clave)
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, Settings
from llama_index.core.node_parser import SemanticSplitterNodeParser
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
import os
from pathlib import Path

# === NUEVO: detectar raíz del proyecto y crear DATA ahí ===
CURRENT_DIR = Path().resolve()
PROJECT_ROOT = CURRENT_DIR.parent       # sube dos niveles desde notebooks/
DATA_DIR = PROJECT_ROOT / "data"

# 1. Crear carpeta y pedir PDFs
if not DATA_DIR.exists():
    DATA_DIR.mkdir(parents=True, exist_ok=True)
    print(f"⚠️ CARPETA 'data' CREADA EN: {DATA_DIR}. Por favor, sube tus PDFs ahí antes de continuar.")
else:
    # 2. Configurar Modelos
    embed_model = OpenAIEmbedding(model="text-embedding-3-small")
    Settings.embed_model = embed_model
    Settings.chunk_size = 1024

    # 3. Configurar Qdrant como almacén
    client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY)
    vector_store = QdrantVectorStore(client=client, collection_name="bio_semantic_db")
    storage_context = StorageContext.from_defaults(vector_store=vector_store)

    # 4. DEFINIR EL SPLITTER SEMÁNTICO (Aquí está la magia)
    splitter = SemanticSplitterNodeParser(
        buffer_size=1,
        breakpoint_percentile_threshold=95,
        embed_model=embed_model
    )

    # 5. Cargar y Procesar
    print("📂 Leyendo PDFs...")
    docs = SimpleDirectoryReader(DATA_DIR).load_data()
    
    print("🧠 Analizando significado y creando chunks...")
    nodes = splitter.get_nodes_from_documents(docs)
    print(f"   -> Se crearon {len(nodes)} chunks semánticos.")

    # 6. Indexar
    print("🚀 Subiendo a Qdrant...")
    index = VectorStoreIndex(nodes, storage_context=storage_context)
    print("✅ ¡Ingesta Lista!")


📂 Leyendo PDFs...
🧠 Analizando significado y creando chunks...


2025-11-24 16:11:47,634 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-24 16:11:49,390 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-24 16:11:50,914 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-24 16:11:54,125 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-24 16:11:55,281 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-24 16:11:57,515 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-24 16:11:58,232 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-24 16:11:59,575 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-24 16:12:01,690 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-24 16:12:03,533 - INFO - HTTP

   -> Se crearon 37 chunks semánticos.
🚀 Subiendo a Qdrant...


2025-11-24 16:12:08,788 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-11-24 16:12:10,454 - INFO - HTTP Request: PUT https://f71299c2-b428-4945-b4c4-eadd84c589da.europe-west3-0.gcp.cloud.qdrant.io:6333/collections/bio_semantic_db "HTTP/1.1 200 OK"
2025-11-24 16:12:10,972 - INFO - HTTP Request: PUT https://f71299c2-b428-4945-b4c4-eadd84c589da.europe-west3-0.gcp.cloud.qdrant.io:6333/collections/bio_semantic_db/index?wait=true "HTTP/1.1 200 OK"
2025-11-24 16:12:11,216 - INFO - HTTP Request: GET https://f71299c2-b428-4945-b4c4-eadd84c589da.europe-west3-0.gcp.cloud.qdrant.io:6333/collections/bio_semantic_db "HTTP/1.1 200 OK"
2025-11-24 16:12:13,375 - INFO - HTTP Request: PUT https://f71299c2-b428-4945-b4c4-eadd84c589da.europe-west3-0.gcp.cloud.qdrant.io:6333/collections/bio_semantic_db/points?wait=true "HTTP/1.1 200 OK"


✅ ¡Ingesta Lista!


### Agente ReAct

In [3]:
import os
from langchain.chat_models import init_chat_model


model = init_chat_model("gpt-4o-mini")

In [2]:
from langchain.agents import create_agent

agent = create_agent(
    model= model,
    tools=[],
    system_prompt="You are a helpful research assistant."
)

In [24]:
result = agent.invoke({
    "messages": [
        {"role": "user", "content": "Hola!"}
    ]
})

2025-11-24 01:45:28,097 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [25]:
result

{'messages': [HumanMessage(content='Hola!', additional_kwargs={}, response_metadata={}, id='8a072b79-5581-4369-a4b5-440eb2559ada'),
  AIMessage(content='¡Hola! ¿Cómo puedo ayudarte hoy?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 20, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_560af6e559', 'id': 'chatcmpl-CfKcAieag9RgQCdhKMOO3PXgOJIZU', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--45fad6b4-6796-422e-9204-b96cb7e84d39-0', usage_metadata={'input_tokens': 20, 'output_tokens': 9, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}